In [193]:
import os
import pandas as pd

main_publisher = 'OYC'

methods = 'embedd-er' #embedd-er or BERT
class_type = 'dct' #dct or rdfs
class_file = class_type + ".csv"

script_dir = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(script_dir, '../Data/' + main_publisher + '/data/')
embeddings_path = os.path.join(script_dir, '../Data/' + main_publisher + '/embeddings/')
precedence_path = os.path.join(script_dir, '../Data/' + main_publisher + '/precedence/')

df_chapters = pd.read_csv(data_path + 'chapters.csv', delimiter = '|')
df_chapters_embeddings = pd.read_csv(embeddings_path + 'chapters_' + methods +'.csv', delimiter = '|', index_col=0)
df_concepts = pd.read_csv(data_path + 'concepts.csv', delimiter = '|')
df_concepts_embeddings = pd.read_csv(embeddings_path + 'concepts_' + methods +'.csv', delimiter = '|', index_col=0)
df_classes = pd.read_csv(data_path + 'classes/rdfs.csv', delimiter = '|')
df_classes_embeddings = pd.read_csv(embeddings_path + '/classes/' + class_type + '_' + methods +'.csv', delimiter = '|')
df_precedences_episodes = pd.read_csv(precedence_path + 'precedences_episodes.csv', delimiter = '|')
df_precedences_series = pd.read_csv(precedence_path + 'precedences_series.csv', delimiter = '|')

df_concepts['Concept'] = df_concepts['Concept'].apply(lambda x : x.split('/')[-1])

df_classes = df_classes.dropna()
print(f'{df_chapters["Cid"].isna().sum().sum():04d} NaN values in chapters.')
print(f'{df_concepts.isna().sum().sum():04d} Nan values in concepts.')
print(f'{df_classes.isna().sum().sum():04d} Nan values in classes.')
print(f'{df_precedences_episodes.isna().sum().sum():04d} Nan values in episdes precedences.')
print(f'{df_precedences_series.isna().sum().sum():04d} Nan values in series precedences.')
print(df_chapters.shape)

0000 NaN values in chapters.
0000 Nan values in concepts.
0000 Nan values in classes.
0000 Nan values in episdes precedences.
0000 Nan values in series precedences.
(2550, 9)


In [194]:
from utils import *

unique_oer_id = id_mapper(df_chapters['Cid'], 'OER')
unique_concept_id =  id_mapper(df_concepts['Concept'], 'Concept')
unique_class_id =  id_mapper(df_classes['Class'], 'Class')

In [195]:
oer_covers_concept_subject = edge_construction(df1 = df_concepts, df2 = unique_oer_id, col = 'mappedID', 
                                       how = 'left', right_on = 'OER')
oer_covers_concept_pr = edge_construction(df1 = df_concepts, df2 = unique_oer_id, col = 'PR', 
                                          how = 'right', right_on = 'OER')
oer_covers_concept_object = edge_construction(df1 = df_concepts, df2 = unique_concept_id, col = 'mappedID', 
                                       how = 'left', right_on = 'Concept')

oer_before_oer_ep_subject = edge_construction(df1 = df_precedences_episodes, df2 = unique_oer_id, col = 'mappedID', 
                                   how = 'left', left_on = 'Before', right_on = 'OER')
oer_before_oer_ep_object = edge_construction(df1 = df_precedences_episodes, df2 = unique_oer_id, col = 'mappedID', 
                                   how = 'left', left_on = 'After', right_on = 'OER')
oer_before_oer_sr_subject = edge_construction(df1 = df_precedences_series, df2 = unique_oer_id, col = 'mappedID', 
                                   how = 'left', left_on = 'Before', right_on = 'OER')
oer_before_oer_sr_object = edge_construction(df1 = df_precedences_series, df2 = unique_oer_id, col = 'mappedID', 
                                   how = 'left', left_on = 'After', right_on = 'OER')

concept_belongs_class_subject = edge_construction(df1 = df_classes, df2 = unique_concept_id, col = 'mappedID', 
                                   how = 'left', left_on = 'Concept', right_on = 'Concept')
concept_belongs_class_object = edge_construction(df1 = df_classes, df2 = unique_class_id, col = 'mappedID', 
                                   how = 'left', left_on = 'Class', right_on = 'Class')

oer_covers_concept = torch.stack([oer_covers_concept_subject, oer_covers_concept_object], dim = 0).long()
oer_covers_concept_rev = torch.stack([oer_covers_concept_object, oer_covers_concept_subject], dim = 0).long()
oer_before_oer_ep = torch.stack([oer_before_oer_ep_subject, oer_before_oer_ep_object], dim = 0).long()
oer_before_oer_sr = torch.stack([oer_before_oer_sr_subject, oer_before_oer_sr_object], dim = 0).long()
concept_belongs_class = torch.stack([concept_belongs_class_subject, concept_belongs_class_object], dim = 0).long()
concept_belongs_class_rev = torch.stack([concept_belongs_class_object, concept_belongs_class_subject], dim = 0).long()
print(oer_covers_concept.shape)
print(oer_covers_concept_rev.shape)
print(oer_before_oer_ep.shape)
print(oer_before_oer_sr.shape)
print(concept_belongs_class.shape)
print(concept_belongs_class_rev.shape)

torch.Size([2, 16786])
torch.Size([2, 16786])
torch.Size([2, 2097])
torch.Size([2, 423])
torch.Size([2, 58295])
torch.Size([2, 58295])


In [197]:
if methods == 'BERT':
    entity_features = 768
elif methods == 'embedd-er':
    entity_features = 300
hidden_channels = [16, 64, 128]
num_layers = [1 ,2, 4, 6]
epochs = [50, 100, 300] #300 the best
learning_rates = [0.001, 0.01, 0.1]

selected_params = [{
    'epochs': epochs_selected, 
    'hidden_channels': hidden_channels_selected, 
    'num_layers': num_layers_selected, #8 is too much => generated NaN values in node attributes
    'lr': learning_rates_selected, 
    'entity_features': entity_features
} for epochs_selected in epochs 
for hidden_channels_selected in hidden_channels
for num_layers_selected in num_layers
for learning_rates_selected in learning_rates]
print(len(selected_params))

108


In [198]:
chapters_embeddings_tmp = {}
concepts_embeddings_tmp = {}
classes_embeddings_tmp = {}

chapters_r = range(len(df_chapters['Cid'].unique()))
concepts_c = range(len(df_concepts['Concept'].unique()))
classes_c = range(len(df_classes['Class'].unique()))

chapters_embeddings = np.zeros(shape=(len(chapters_r), entity_features))
concepts_embeddings = np.zeros(shape=(len(concepts_c), entity_features))
classes_embeddings = np.zeros(shape=(len(classes_c), entity_features))


i = 0
for r in chapters_r:
    chapters_embeddings_tmp[r] = list(filter(None, df_chapters_embeddings['Chapters Embeddings'][r].strip("[]\n").replace("'","").split(" ")))
    chapters_embeddings_tmp[r] = [float(f) for f in chapters_embeddings_tmp[r]]
    for a in range(len(chapters_embeddings_tmp[r])):
            chapters_embeddings[i][a] = chapters_embeddings_tmp[r][a]
    i += 1

i = 0
for r in concepts_c:
    concepts_embeddings_tmp[r] = list(filter(None, df_concepts_embeddings['Concepts Embeddings'][r].strip("[]\n").replace("'","").split(" ")))
    concepts_embeddings_tmp[r] = [float(f) for f in concepts_embeddings_tmp[r]]
    for a in range(len(concepts_embeddings_tmp[r])):
            concepts_embeddings[i][a] = concepts_embeddings_tmp[r][a]
    i += 1   

i = 0
for r in classes_c:
    classes_embeddings_tmp[r] = list(filter(None, df_classes_embeddings['Classes Embeddings'][r].strip("[]\n").replace("'","").split(" ")))
    classes_embeddings_tmp[r] = [float(f) for f in classes_embeddings_tmp[r]]
    for a in range(len(classes_embeddings_tmp[r])):
            classes_embeddings[i][a] = classes_embeddings_tmp[r][a]
    i += 1

chapters_embeddings = torch.from_numpy(chapters_embeddings).to(torch.float32)
concepts_embeddings = torch.from_numpy(concepts_embeddings).to(torch.float32)
classes_embeddings = torch.from_numpy(classes_embeddings).to(torch.float32)

In [199]:
chapters_embeddings.shape

torch.Size([2550, 300])

In [200]:
import random

def seed_everything(seed=0):                                                  
       random.seed(seed)                                                            
       torch.manual_seed(seed)                                                      
       torch.cuda.manual_seed_all(seed)                                             
       np.random.seed(seed)                                                         
       os.environ['PYTHONHASHSEED'] = str(seed)                                     
       torch.backends.cudnn.deterministic = True                                    
       torch.backends.cudnn.benchmark = False

In [201]:
import sys
sys.path.insert(1, '../Models/')

In [202]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
data['OER'].node_id = torch.tensor(unique_oer_id['mappedID'].values)
data['OER'].x = chapters_embeddings
data['Concept'].node_id = torch.tensor(unique_concept_id['mappedID'].values)
data['Concept'].x = concepts_embeddings
data['Class'].node_id = torch.tensor(unique_class_id['mappedID'].values)
data['Class'].x = classes_embeddings
data['OER', 'covers', 'Concept'].edge_index = oer_covers_concept
data['Concept', 'rev_covers', 'OER'].edge_index = oer_covers_concept_rev

data['OER', 'covers', 'Concept'].edge_attr = oer_covers_concept_pr
#print(oer_before_oer_ep.shape)
data['OER', 'before_sr', 'OER'].edge_index = oer_before_oer_sr
data['OER', 'before_ep', 'OER'].edge_index = oer_before_oer_ep
data['Concept', 'belongs', 'Class'].edge_index = concept_belongs_class
data['Class', 'rev_belongs', 'Concept'].edge_index = concept_belongs_class_rev

#data = T.ToUndirected()(data)
data.validate()
print(data)

HeteroData(
  OER={
    node_id=[2550],
    x=[2550, 300]
  },
  Concept={
    node_id=[6007],
    x=[6007, 300]
  },
  Class={
    node_id=[292],
    x=[292, 300]
  },
  (OER, covers, Concept)={
    edge_index=[2, 16786],
    edge_attr=[16830]
  },
  (Concept, rev_covers, OER)={ edge_index=[2, 16786] },
  (OER, before_sr, OER)={ edge_index=[2, 423] },
  (OER, before_ep, OER)={ edge_index=[2, 2097] },
  (Concept, belongs, Class)={ edge_index=[2, 58295] },
  (Class, rev_belongs, Concept)={ edge_index=[2, 58295] }
)


In [203]:
agnostic = False
if agnostic:
    num_val = 0.5
    num_test = 0.5
else:
    num_val = 0.1
    num_test = 0.1
seed_everything()
transform = T.RandomLinkSplit(
    num_val = num_val,
    num_test = num_test,
    disjoint_train_ratio = 0.0,
    neg_sampling_ratio = 0.8,
    add_negative_train_samples = True,
    edge_types=('OER', 'before_sr', 'OER')
)

train_data, val_data, test_data = transform(data)
print(f'{len(train_data["OER", "before_sr", "OER"].edge_label.detach().numpy())}\t Edges for training')
print(f'{len(val_data["OER", "before_sr", "OER"].edge_label.detach().numpy())}\t Edges for validation')
print(f'{len(test_data["OER", "before_sr", "OER"].edge_label.detach().numpy())}\t Edges for testing')
print(train_data["OER", "before_sr", "OER"].edge_label_index[0][:5])
print(val_data["OER", "before_sr", "OER"].edge_label_index[0][:5])
print(test_data["OER", "before_sr", "OER"].edge_label_index[0][:5])
print(train_data["OER", "before_sr", "OER"].edge_label_index[1][:5])
print(val_data["OER", "before_sr", "OER"].edge_label_index[1][:5])
print(test_data["OER", "before_sr", "OER"].edge_label_index[1][:5])

610	 Edges for training
75	 Edges for validation
75	 Edges for testing
tensor([1692, 1554, 2541, 1480,  422])
tensor([2187,  412, 1881, 1991,  541])
tensor([2529,  769, 2363, 1027, 1958])
tensor([1693, 1555, 2542, 1481,  423])
tensor([2188,  413, 1882, 1992,  542])
tensor([2530,  770, 2364, 1028, 1959])


In [204]:
seed_everything()
cross_val_data = {}
cross_val_data["OER", "before_sr", "OER"] = {}
print(len(train_data["OER", "before_sr", "OER"].edge_label_index[0]) + 
      len(val_data["OER", "before_sr", "OER"].edge_label_index[0]) +
      len(test_data["OER", "before_sr", "OER"].edge_label_index[0]))

cross_val_data["OER", "before_sr", "OER"]["edge_label"] = torch.cat(
    [train_data["OER", "before_sr", "OER"].edge_label, 
     val_data["OER", "before_sr", "OER"].edge_label,
     test_data["OER", "before_sr", "OER"].edge_label], 
    dim = 0).long()
cross_val_data["OER", "before_sr", "OER"]["edge_label_index"] = torch.cat(
    [train_data["OER", "before_sr", "OER"].edge_label_index, 
     val_data["OER", "before_sr", "OER"].edge_label_index,
     test_data["OER", "before_sr", "OER"].edge_label_index], 
    dim = 1).long()
cross_val_data["OER", "before_sr", "OER"]["edge_index"] = torch.cat(
    [train_data["OER", "before_sr", "OER"].edge_index, 
     val_data["OER", "before_sr", "OER"].edge_index,
     test_data["OER", "before_sr", "OER"].edge_index],
    dim = 1).long()
print(len(cross_val_data["OER", "before_sr", "OER"]["edge_label"]))
print(len(cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][0]))
print(len(cross_val_data["OER", "before_sr", "OER"]["edge_index"][0]))

760
760
760
1059


In [205]:
seed_everything()
num_samples = len(cross_val_data["OER", "before_sr", "OER"]["edge_label"])
shuffled_index = np.arange(num_samples)
np.random.shuffle(shuffled_index)
cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][0] = cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][0][shuffled_index]
cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][1] = cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][1][shuffled_index]
cross_val_data["OER", "before_sr", "OER"]["edge_label"] = cross_val_data["OER", "before_sr", "OER"]["edge_label"][shuffled_index]

In [206]:
cv = 5
chunk_size = int(len(cross_val_data["OER", "before_sr", "OER"]["edge_label"]) / cv)
cross_val_chunks = []
for n in range(cv):
    cross_val_chunk = {}
    cross_val_chunk["OER", "before_sr", "OER"] = {}
    cross_val_chunk["OER", "before_sr", "OER"]["edge_label"] = {}
    begin = n * chunk_size
    if n == cv - 1:
        end = len(cross_val_data["OER", "before_sr", "OER"]["edge_label"])
    else :
        end = (n+1) * chunk_size
    cross_val_chunk["OER", "before_sr", "OER"]["edge_label"] = cross_val_data["OER", "before_sr", "OER"]["edge_label"][begin : end]
    cross_val_chunk["OER", "before_sr", "OER"]["edge_label_index"] = []
    cross_val_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][0][begin : end])
    cross_val_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(cross_val_data["OER", "before_sr", "OER"]["edge_label_index"][1][begin : end])
    cross_val_chunks.append(cross_val_chunk)

In [207]:
cross_val_data_train = []
cross_val_data_test = []
for n in range(cv) :
    cross_val_data_train_chunk = {}
    cross_val_data_train_chunk["OER", "before_sr", "OER"] = {}
    cross_val_data_train_chunk["OER", "before_sr", "OER"]["edge_label"] = torch.cat(
    [cross_val_chunks[(n+1)%cv]["OER", "before_sr", "OER"]["edge_label"], 
     cross_val_chunks[(n+2)%cv]["OER", "before_sr", "OER"]["edge_label"],
     cross_val_chunks[(n+3)%cv]["OER", "before_sr", "OER"]["edge_label"],
     cross_val_chunks[(n+4)%cv]["OER", "before_sr", "OER"]["edge_label"]
     ], 
    dim = 0).long()
    cross_val_data_test_chunk = {}
    cross_val_data_test_chunk["OER", "before_sr", "OER"] = {}
    cross_val_data_test_chunk["OER", "before_sr", "OER"]["edge_label"] = cross_val_chunks[n]["OER", "before_sr", "OER"]["edge_label"]

    cross_val_data_train_chunk["OER", "before_sr", "OER"]["edge_label_index"] = []
    cross_val_data_train_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(torch.cat(
    [cross_val_chunks[(n+1)%cv]["OER", "before_sr", "OER"]["edge_label_index"][0], 
     cross_val_chunks[(n+2)%cv]["OER", "before_sr", "OER"]["edge_label_index"][0],
     cross_val_chunks[(n+3)%cv]["OER", "before_sr", "OER"]["edge_label_index"][0],
     cross_val_chunks[(n+4)%cv]["OER", "before_sr", "OER"]["edge_label_index"][0]
     ], 
    dim = 0).long())
    cross_val_data_train_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(torch.cat(
    [cross_val_chunks[(n+1)%cv]["OER", "before_sr", "OER"]["edge_label_index"][1], 
     cross_val_chunks[(n+2)%cv]["OER", "before_sr", "OER"]["edge_label_index"][1],
     cross_val_chunks[(n+3)%cv]["OER", "before_sr", "OER"]["edge_label_index"][1],
     cross_val_chunks[(n+4)%cv]["OER", "before_sr", "OER"]["edge_label_index"][1]
     ], 
    dim = 0).long())
    cross_val_data_test_chunk["OER", "before_sr", "OER"]["edge_label_index"] = []
    cross_val_data_test_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(
        cross_val_chunks[n]["OER", "before_sr", "OER"]["edge_label_index"][0])
    cross_val_data_test_chunk["OER", "before_sr", "OER"]["edge_label_index"].append(
        cross_val_chunks[n]["OER", "before_sr", "OER"]["edge_label_index"][1])
    
    cross_val_data_train.append(cross_val_data_train_chunk)
    cross_val_data_test.append(cross_val_data_test_chunk)

len(cross_val_data_train)

5

In [208]:
from models import ModelReinjection

df_ModelReinjection = pd.DataFrame()
seed_everything()
for params in selected_params:
    for n in range(cv) :
        train_data["OER", "before_sr", "OER"].edge_label = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label"]
        train_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label_index"]
        test_data["OER", "before_sr", "OER"].edge_label = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label"]
        test_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label_index"]
        
        results = {}

        model = ModelReinjection(node_types = data.node_types, heads = 3, hidden_channels = params['hidden_channels'], entity_features = params['entity_features'], out_channels = 1, num_layers = params['num_layers'])
        start = time.time()
        train_results = train(model, train_data, params['epochs'], params['lr'])
        end = time.time()
        model = train_results['Model']
        results['Epochs'] = params['epochs']
        results['Learning rate'] = params['lr']
        results['Layers'] = params['num_layers']
        results['Channels'] = params['hidden_channels']
        results['Time'] = end - start
        results['Embedding'] = methods
        results['Class'] = class_type

        test_results = predict(model, test_data)
        results['Test Precision'] = test_results["Precision"]
        results['Test Accuracy'] = test_results["Accuracy"]
        results['Test Recall'] = test_results["Recall"]
        results['Test F1'] = test_results["F1"]

        loss_values = train_results['Loss_values']
        results['Loss'] = train_results['Loss']
        results['Duration'] = train_results['Duration']
        df_ModelReinjection = pd.concat([df_ModelReinjection, pd.DataFrame([results])], ignore_index = True)
df_ModelReinjection.head(20)

In [ ]:
print(round(df_ModelReinjection["Test Accuracy"].values.min(), 2))
print(round(df_ModelReinjection["Test Accuracy"].values.mean(), 2))
print(round(df_ModelReinjection["Test Accuracy"].values.max(), 2))

0.74
0.86
0.92


In [ ]:
from models import ModelSAGE

df_ModelSAGE = pd.DataFrame()
seed_everything()
for params in selected_params:
    for n in range(cv) :
        train_data["OER", "before_sr", "OER"].edge_label = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label"]
        train_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label_index"]
        test_data["OER", "before_sr", "OER"].edge_label = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label"]
        test_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label_index"]
        
        results = {}

        model = ModelSAGE(node_types = data.node_types, heads = 3, hidden_channels = params['hidden_channels'], entity_features = params['entity_features'], out_channels = 1, num_layers = params['num_layers'])
        start = time.time()
        train_results = train(model, train_data, params['epochs'], params['lr'])
        end = time.time()
        model = train_results['Model']
        results['Epochs'] = params['epochs']
        results['Learning rate'] = params['lr']
        results['Layers'] = params['num_layers']
        results['Channels'] = params['hidden_channels']
        results['Time'] = end - start
        results['Embedding'] = methods
        results['Class'] = class_type
        

        test_results = predict(model, test_data)
        results['Test Precision'] = test_results["Precision"]
        results['Test Accuracy'] = test_results["Accuracy"]
        results['Test Recall'] = test_results["Recall"]
        results['Test F1'] = test_results["F1"]

        loss_values = train_results['Loss_values']
        results['Loss'] = train_results['Loss']
        results['Duration'] = train_results['Duration']
        df_ModelSAGE = pd.concat([df_ModelSAGE, pd.DataFrame([results])], ignore_index = True)
df_ModelSAGE.head(20)

,Epochs,Learning rate,Layers,Channels,Time,Embedding,Class,Test Precision,Test Accuracy,Test Recall,Test F1,Loss,Duration
0,50,0.001,1,16,1.170987,embedd-er,dct,0.852792,0.856338,0.854226,0.855161,8.274610,1.169800
1,50,0.001,1,16,0.960906,embedd-er,dct,0.893401,0.887324,0.885657,0.886087,8.778843,0.960053
2,50,0.001,1,16,0.988908,embedd-er,dct,0.935829,0.867606,0.875287,0.865726,8.149256,0.987559
3,50,0.001,1,16,1.097312,embedd-er,dct,0.868687,0.870423,0.868162,0.869151,8.236922,1.096084
4,50,0.001,1,16,1.111209,embedd-er,dct,0.871795,0.843137,0.842757,0.841243,8.818679,1.109798
5,50,0.010,1,16,1.098330,embedd-er,dct,0.844221,0.850704,0.848166,0.849322,8.075301,1.097117
6,50,0.010,1,16,1.078934,embedd-er,dct,0.904762,0.881690,0.882212,0.880895,7.650656,1.077572
7,50,0.010,1,16,1.175849,embedd-er,dct,0.916667,0.887324,0.886617,0.884123,7.117864,1.174298
8,50,0.010,1,16,1.165003,embedd-er,dct,0.874396,0.895775,0.891600,0.894149,6.982259,1.163843
9,50,0.010,1,16,1.150537,embedd-er,dct,0.873171,0.865546,0.861940,0.862965,5.193651,1.149393


In [ ]:
print(round(df_ModelSAGE["Test Accuracy"].values.min(), 2))
print(round(df_ModelSAGE["Test Accuracy"].values.mean(), 2))
print(round(df_ModelSAGE["Test Accuracy"].values.max(), 2))

0.74
0.85
0.91


In [ ]:
from models import ModelGAT

df_ModelGAT = pd.DataFrame()
v = 'v8'
seed_everything()
for params in selected_params:
    for n in range(cv) :
        train_data["OER", "before_sr", "OER"].edge_label = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label"]
        train_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label_index"]
        test_data["OER", "before_sr", "OER"].edge_label = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label"]
        test_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label_index"]
        
        results = {}

        model = ModelGAT(node_types = data.node_types, heads = 3, hidden_channels = params['hidden_channels'], entity_features = params['entity_features'], out_channels = 1, num_layers = params['num_layers'])
        start = time.time()
        train_results = train(model, train_data, params['epochs'], params['lr'])
        end = time.time()
        model = train_results['Model']
        results['Epochs'] = params['epochs']
        results['Learning rate'] = params['lr']
        results['Layers'] = params['num_layers']
        results['Channels'] = params['hidden_channels']
        results['Time'] = end - start
        results['Embedding'] = methods
        results['Class'] = class_type
        
        '''validation_results = predict(model, val_data)
        #results['Validation AUC'] = validation_results["AUC"]
        results['Validation Precision'] = validation_results["Precision"]
        results['Validation Accuracy'] = validation_results["Accuracy"]
        results['Validation Recall'] = validation_results["Recall"]
        results['Validation F1'] = validation_results["F1"]'''

        test_results = predict(model, test_data)
        #results['Test AUC'] = test_results["AUC"]
        results['Test Precision'] = test_results["Precision"]
        results['Test Accuracy'] = test_results["Accuracy"]
        results['Test Recall'] = test_results["Recall"]
        results['Test F1'] = test_results["F1"]

        loss_values = train_results['Loss_values']
        results['Loss'] = train_results['Loss']
        results['Duration'] = train_results['Duration']
        df_ModelGAT = pd.concat([df_ModelGAT, pd.DataFrame([results])], ignore_index = True)
df_ModelGAT.head(20)

,Epochs,Learning rate,Layers,Channels,Time,Embedding,Class,Test Precision,Test Accuracy,Test Recall,Test F1,Loss,Duration
0,50,0.001,1,16,2.089982,embedd-er,dct,0.811111,0.780282,0.781180,0.779889,7.046273,2.088793
1,50,0.001,1,16,2.207049,embedd-er,dct,0.807018,0.746479,0.750721,0.746235,8.328095,2.205931
2,50,0.001,1,16,2.134769,embedd-er,dct,0.862857,0.766197,0.776765,0.764515,6.961636,2.132852
3,50,0.001,1,16,2.131584,embedd-er,dct,0.837079,0.797183,0.799064,0.796820,8.212745,2.130558
4,50,0.001,1,16,2.082735,embedd-er,dct,0.856354,0.798319,0.802239,0.797325,6.815372,2.081699
5,50,0.010,1,16,2.120353,embedd-er,dct,0.851282,0.850704,0.848963,0.849630,4.843059,2.119279
6,50,0.010,1,16,2.051335,embedd-er,dct,0.841026,0.825352,0.823638,0.823638,5.211226,2.050384
7,50,0.010,1,16,1.868572,embedd-er,dct,0.898477,0.850704,0.852463,0.847431,5.694570,1.867584
8,50,0.010,1,16,1.876536,embedd-er,dct,0.837696,0.822535,0.821575,0.821424,5.418075,1.875400
9,50,0.010,1,16,2.245638,embedd-er,dct,0.858586,0.834734,0.833142,0.832414,5.371946,2.244507


In [ ]:
print(round(df_ModelGAT["Test Accuracy"].values.min(), 2))
print(round(df_ModelGAT["Test Accuracy"].values.mean(), 2))
print(round(df_ModelGAT["Test Accuracy"].values.max(), 2))

0.41
0.79
0.89


In [ ]:
from models import ModelConv

df_ModelConv = pd.DataFrame()
seed_everything()
for params in selected_params:
    for n in range(cv) :
        train_data["OER", "before_sr", "OER"].edge_label = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label"]
        train_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_train[n]["OER", "before_sr", "OER"]["edge_label_index"]
        test_data["OER", "before_sr", "OER"].edge_label = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label"]
        test_data["OER", "before_sr", "OER"].edge_label_index = cross_val_data_test[n]["OER", "before_sr", "OER"]["edge_label_index"]
        
        results = {}

        model = ModelConv(node_types = data.node_types, heads = 1, hidden_channels = params['hidden_channels'], entity_features = params['entity_features'], out_channels = 1, num_layers = params['num_layers'])
        start = time.time()
        train_results = train(model, train_data, params['epochs'], params['lr'])
        end = time.time()
        model = train_results['Model']
        results['Epochs'] = params['epochs']
        results['Learning rate'] = params['lr']
        results['Layers'] = params['num_layers']
        results['Channels'] = params['hidden_channels']
        results['Time'] = end - start
        results['Embedding'] = methods
        results['Class'] = class_type

        test_results = predict(model, test_data)
        results['Test Precision'] = test_results["Precision"]
        results['Test Accuracy'] = test_results["Accuracy"]
        results['Test Recall'] = test_results["Recall"]
        results['Test F1'] = test_results["F1"]

        loss_values = train_results['Loss_values']
        results['Loss'] = train_results['Loss']
        results['Duration'] = train_results['Duration']
        df_ModelConv = pd.concat([df_ModelConv, pd.DataFrame([results])], ignore_index = True)
df_ModelConv.head(20)

,Epochs,Learning rate,Layers,Channels,Time,Embedding,Class,Test Precision,Test Accuracy,Test Recall,Test F1,Loss,Duration
0,50,0.001,1,16,1.018935,embedd-er,dct,0.793103,0.752113,0.754067,0.751953,9.603217,1.018116
1,50,0.001,1,16,1.034443,embedd-er,dct,0.793478,0.754930,0.755609,0.753804,9.885407,1.033553
2,50,0.001,1,16,1.038302,embedd-er,dct,0.841808,0.749296,0.758210,0.747242,9.037477,1.037385
3,50,0.001,1,16,1.045387,embedd-er,dct,0.800000,0.763380,0.764571,0.762836,9.301456,1.044595
4,50,0.001,1,16,1.042555,embedd-er,dct,0.801136,0.733894,0.738567,0.733056,9.759918,1.041748
5,50,0.010,1,16,1.045957,embedd-er,dct,0.776596,0.757746,0.756938,0.756725,11.442283,1.044980
6,50,0.010,1,16,1.030128,embedd-er,dct,0.803109,0.780282,0.778686,0.778367,13.464461,1.029147
7,50,0.010,1,16,1.020785,embedd-er,dct,0.832461,0.766197,0.768227,0.762205,11.047801,1.019906
8,50,0.010,1,16,0.967407,embedd-er,dct,0.809756,0.816901,0.812660,0.814302,10.608821,0.966455
9,50,0.010,1,16,0.833351,embedd-er,dct,0.784038,0.775910,0.767987,0.770048,12.159465,0.832652


In [ ]:
print(round(df_ModelConv["Test Accuracy"].values.min(), 2))
print(round(df_ModelConv["Test Accuracy"].values.mean(), 2))
print(round(df_ModelConv["Test Accuracy"].values.max(), 2))

0.53
0.77
0.87


In [ ]:
df_ModelReinjection['Model'] = 'Reinjection'
df_ModelConv['Model'] = 'Conv'
df_ModelGAT['Model'] = 'GAT'
df_ModelSAGE['Model'] = 'SAGE'

df_ModelReinjection['Data'] = main_publisher
df_ModelConv['Data'] = main_publisher
df_ModelGAT['Data'] = main_publisher
df_ModelSAGE['Data'] = main_publisher

In [ ]:
df_ModelsAll = pd.concat([df_ModelReinjection, df_ModelConv, df_ModelGAT, df_ModelSAGE])
df_ModelsAll.to_csv(f'../Output/Ablation/results_{main_publisher}_{methods}_{class_type}_bis.csv', index = False)